In [70]:
import numpy as np
import pandas as pd
import random
import glob
import math
import xgboost
from sklearn import preprocessing

from matplotlib import pyplot


In [71]:
# Get list of files for the Geo grouped data
# Data sourced from https://data.cms.gov/provider-summary-by-type-of-service/medicare-physician-other-practitioners/medicare-physician-other-practitioners-by-geography-and-service
data_files_list = glob.glob('data/MUP*Geo.csv')
data_files_list 

# For training use 2013 to 2016 to predict 2017
# For testing use 2014 to 2017 to predict 2018
# for validation use 2015 to 2018 to predict 2019

# For presentation use 2016 to 2019 to predict 2020 (and compare against 2019)

['data/MUP_PHY_R19_P04_V10_D17_Geo.csv',
 'data/MUP_PHY_R20_P04_V10_D18_Geo.csv',
 'data/MUP_PHY_R19_P04_V10_D13_Geo.csv',
 'data/MUP_PHY_R19_P04_V10_D15_Geo.csv',
 'data/MUP_PHY_R19_P04_V10_D16_Geo.csv',
 'data/MUP_PHY_R21_P04_V10_D19_Geo.csv',
 'data/MUP_PHY_R19_P04_V10_D14_Geo.csv']

In [72]:
df_geo_full = []
for _file in data_files_list:
    df_geo = pd.read_csv(
         _file,
         header=0, 
         #skiprows=lambda i: i>0 and random.random() > p
    )
    year= _file[-10:-8]
    df_geo['year'] = 2000+int(year)
    if len(df_geo_full)!=0:
        df_geo_full = df_geo_full.append(df_geo, ignore_index=False)
    else:
        df_geo_full = df_geo.copy()

df_geo_full= df_geo_full.loc[df_geo_full['year']!= 0,:]

excl_states =  ['Armed Forces Central/South America', 'Armed Forces Europe', 'Armed Forces Pacific', 'Foreign Country','National']
df_geo_full = df_geo_full.loc[~df_geo_full['Rndrng_Prvdr_Geo_Desc'].isin(excl_states),:].copy()

/tmp/ipykernel_19469/1583512775.py:3: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  df_geo = pd.read_csv(
/tmp/ipykernel_19469/1583512775.py:3: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  df_geo = pd.read_csv(
/tmp/ipykernel_19469/1583512775.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_geo_full = df_geo_full.append(df_geo, ignore_index=False)
/tmp/ipykernel_19469/1583512775.py:3: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  df_geo = pd.read_csv(
/tmp/ipykernel_19469/1583512775.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_geo_full = df_geo_full.append(df_geo, ignore_index=False)
/tmp/ipykernel_19469/1583512775.py:3: DtypeW

In [73]:
# Prep geo_full for train/test/val

df_geo_features = df_geo_full.loc[:,['Rndrng_Prvdr_Geo_Desc', 'HCPCS_Cd', 'Tot_Srvcs','year']].copy()
df_geo_features = df_geo_features.groupby(['Rndrng_Prvdr_Geo_Desc', 'HCPCS_Cd','year'])[ 'Tot_Srvcs'].sum().reset_index()
df_geo_features.loc[:,'Tot_Srvcs'] = df_geo_features['Tot_Srvcs'].apply(lambda x: math.log(x))

df_geo_features.loc[:,'idx'] = df_geo_features.apply(lambda x:  "_".join([str(x['Rndrng_Prvdr_Geo_Desc']), str(x['HCPCS_Cd'])]), axis=1)
start_year = 2013
no_of_years = 4
df_geo_features.loc[:,'year'] = start_year + no_of_years - df_geo_features['year'] 
df_geo_features_train = df_geo_features.loc[df_geo_features['year']>=0,:].copy()
df_geo_features_train = df_geo_features_train.loc[df_geo_features_train['year'] <= no_of_years,:]

df_geo_features_test = df_geo_features.loc[df_geo_features['year']>=-1,:].copy()
df_geo_features_test.loc[:,'year'] = df_geo_features_test['year'] + 1
df_geo_features_test = df_geo_features_test.loc[df_geo_features_test['year']<=no_of_years,:].copy()

df_geo_features_train = df_geo_features_train.pivot(index=['idx'], columns=['year'], values='Tot_Srvcs')
df_geo_features_test = df_geo_features_test.pivot(index=['idx'], columns=['year'], values='Tot_Srvcs')

col_map = {
    0: "target",
    1: "1",
    2: "2",
    3: "3"
}

df_geo_features_train.rename(columns = col_map, inplace=True)
df_geo_features_test.rename(columns = col_map, inplace=True)

df_geo_features_train['state'] = df_geo_features_train.reset_index()['idx'].apply(lambda x: x.split("_")[0]).values
df_geo_features_test['state'] = df_geo_features_test.reset_index()['idx'].apply(lambda x: x.split("_")[0]).values

df_geo_features_train.reset_index(inplace=True)
df_geo_features_test.reset_index(inplace=True)

df_geo_features_train.fillna(0, inplace=True)
df_geo_features_test.fillna(0, inplace=True)

In [74]:
df_geo_features_test.sample(5)

year,idx,target,1,2,3,4,state
150984,New York_33463,4.564348,3.891820,4.465908,4.127134,4.127134,New York
214365,Utah_33235,3.931826,3.555348,3.496508,3.496508,2.944439,Utah
240699,Wyoming_63709,0.000000,0.000000,0.000000,0.000000,2.564949,Wyoming
125952,Montana_29819,0.000000,0.000000,2.833213,2.484907,0.000000,Montana
123014,Missouri_72020,8.843038,8.824089,8.792398,8.764834,8.856946,Missouri


In [75]:
df_geo_features_train.sample(5)

year,idx,target,1,2,3,4,state
207088,Texas_33418,5.726848,5.308268,4.804021,0.000000,0.000000,Texas
105083,Michigan_19281,7.169350,7.185387,7.202661,7.216709,0.000000,Michigan
6226,Alaska_70491,6.150603,6.186209,6.232448,5.866468,5.783825,Alaska
216388,Vermont_10030,0.000000,2.484907,2.833213,3.178054,0.000000,Vermont
109104,Michigan_87177,6.842683,7.003065,6.993933,7.295735,7.484930,Michigan


In [76]:
# Import population estimates data
pop_est = pd.read_csv("data/SC-EST2020-ALLDATA5.csv")

# Drop columns that is not needed for "one row" melting
pop_est.drop(['SUMLEV', 'REGION', 'DIVISION', 'STATE'], axis='columns', inplace=True)

In [77]:
pop_est.sample(3)

,NAME,SEX,ORIGIN,RACE,AGE,CENSUS2010POP,ESTIMATESBASE2010,POPESTIMATE2010,POPESTIMATE2011,POPESTIMATE2012,POPESTIMATE2013,POPESTIMATE2014,POPESTIMATE2015,POPESTIMATE2016,POPESTIMATE2017,POPESTIMATE2018,POPESTIMATE2019,POPESTIMATE2020
1990,Alabama,1,1,4,12,534,534,523,530,580,580,594,636,671,607,658,631,667
161784,South Dakota,2,1,2,18,135,135,136,134,155,156,159,174,178,194,178,190,215
133931,North Dakota,1,2,3,29,17,17,18,13,5,21,42,33,27,18,26,42,40


In [78]:
pop_est['age_category'] =  pop_est['AGE'].apply(lambda x: 1 if x >40  else 0) 
#pop_est['idx'] = pop_est.apply(lambda x:  "_".join([str(x['SEX']), str(x['ORIGIN']), str(x['RACE']), str(x['AGE'])]), axis=1)
pop_est.drop(['SEX', 'ORIGIN', 'RACE', 'AGE'], axis='columns', inplace=True)

In [79]:
pop_agg_df = pop_est.copy()
pop_agg_df.drop(['CENSUS2010POP','ESTIMATESBASE2010'], axis = 'columns', inplace=True)

pop_agg_df = pd.melt(pop_agg_df,id_vars=['NAME','age_category'])

pop_agg_df.loc[:,'year'] = pop_agg_df['variable'].apply(lambda x: int(x[-4:]))
#popestimate df
pop_agg_df.drop(['variable'], axis='columns', inplace=True)

pop_agg_df = pop_agg_df.groupby(['NAME','age_category','year']).sum().reset_index()
# set t=0 for base year

# set year count to integers from 0 to x for train and test split
start_year = 2013
no_of_years = 4
pop_agg_df.loc[:,'year'] = start_year + no_of_years - pop_agg_df['year'] 
pop_train_df = pop_agg_df.loc[pop_agg_df['year']>0,:].copy()
pop_train_df = pop_train_df.loc[pop_train_df['year'] <= no_of_years,:]
pop_test_df = pop_agg_df.loc[pop_agg_df['year']>-1,:].copy()
pop_test_df.loc[:,'year'] = pop_test_df['year'] + 1
pop_test_df = pop_test_df.loc[pop_test_df['year'] <= no_of_years,:]

pop_train_df.loc[:,'idx'] = pop_train_df.apply(lambda x:  "_".join([str(x['age_category']), str(x['year'])]), axis=1)
pop_train_df = pop_train_df.pivot(index='NAME', columns=['idx'], values='value')

pop_test_df.loc[:,'idx'] = pop_test_df.apply(lambda x:  "_".join([str(x['age_category']), str(x['year'])]), axis=1)
pop_test_df = pop_test_df.pivot(index='NAME', columns=['idx'], values='value')

# normalise values based on train data only

_mean = pop_train_df.stack().mean()
_std = pop_train_df.stack().std()

pop_train_df = (pop_train_df-_mean)/_std
pop_test_df = (pop_test_df-_mean)/_std

pop_train_df.index.name = 'state'
pop_test_df.index.name = 'state'

pop_train_df.reset_index(inplace=True)
pop_test_df.reset_index(inplace=True)

In [80]:
pop_train_df.sample(3)

idx,state,0_1,0_2,0_3,0_4,1_1,1_2,1_3,1_4
46,Virginia,0.410374,0.404800,0.398453,0.391431,0.197277,0.186812,0.176805,0.165156
50,Wyoming,-0.781783,-0.781225,-0.781926,-0.782201,-0.798755,-0.799054,-0.799364,-0.799324
15,Iowa,-0.403392,-0.405209,-0.408003,-0.411965,-0.474962,-0.476538,-0.477528,-0.478731


In [81]:
pop_test_df.sample(3)

idx,state,0_1,0_2,0_3,0_4,1_1,1_2,1_3,1_4
40,South Carolina,-0.143067,-0.150215,-0.157097,-0.165300,-0.211951,-0.223221,-0.235576,-0.247250
3,Arkansas,-0.422374,-0.423939,-0.425101,-0.426399,-0.489533,-0.491847,-0.494542,-0.496743
6,Connecticut,-0.366636,-0.366110,-0.364265,-0.362967,-0.383384,-0.383339,-0.383399,-0.383345


In [82]:
full_train = df_geo_features_train.merge(pop_train_df, on='state',how='left').drop(['state'], axis = 1).set_index('idx')
full_test = df_geo_features_test.merge(pop_test_df, on='state',how='left').drop(['state'], axis = 1).set_index('idx')

In [83]:
full_test.sample(5)

,target,1,2,3,4,0_1,0_2,0_3,0_4,1_1,1_2,1_3,1_4
idx,,,,,,,,,,,,,
Mississippi_86644,2.708050,0.000000,0.000000,0.000000,0.000000,-0.421682,-0.420003,-0.418299,-0.416519,-0.500592,-0.502625,-0.504733,-0.506360
Missouri_46060,2.708050,3.135494,2.708050,2.639057,3.433987,0.036166,0.032899,0.031488,0.028760,-0.084719,-0.088102,-0.092177,-0.095076
Kansas_73220,5.147494,5.093750,4.962845,4.962845,4.700480,-0.413324,-0.412155,-0.411754,-0.413781,-0.516354,-0.517500,-0.518983,-0.519892
Florida_29899,2.564949,2.708050,0.000000,0.000000,2.944439,1.995818,1.953507,1.903172,1.856782,2.066368,2.009058,1.943417,1.885768
South Dakota_37248,2.944439,3.218876,0.000000,0.000000,0.000000,-0.736274,-0.738188,-0.739969,-0.740969,-0.763757,-0.764751,-0.765655,-0.766213


In [84]:
best_params = {'colsample_bytree': 0.9682038820754064,
 'gamma': 1.0184993645158578,
 'max_depth': int(12.0),
 'min_child_weight': int(8.0),
 'n_estimators': int(6),
 'reg_alpha': 113.0,
 'reg_lambda': 0.08792023426025397}

model = xgboost.XGBRegressor(colsample_bytree=0.4,
                 gamma=1.0184993645158578,                 
                 learning_rate=0.007,
                 max_depth=12,
                 min_child_weight=8,
                 n_estimators=6000,                                                                    
                 reg_alpha=0,
                 reg_lambda=0,
                 subsample=0.8,
                 seed=117,
                            ) 


In [85]:
x_train = full_train.drop(["target"], axis=1)
y_train = full_train['target']
x_test = full_test.drop(["target"], axis=1)
y_test= full_test['target']

In [ ]:
model.fit(x_train, y_train, eval_set=[(x_train, y_train),(x_test,y_test)], early_stopping_rounds=50,verbose=False)

/home/deepracer/MADS-Medicare-Project/venv/lib/python3.8/site-packages/xgboost/data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


In [ ]:
results = model.evals_result()
# plot learning curves
_train_results = np.array(list(map(math.exp,results['validation_0']['rmse'])))
_test_results = np.array(list(map(math.exp,results['validation_1']['rmse'])))
pyplot.plot(_train_results, label='train')
pyplot.plot(_test_results, label='test')
# show the legend
pyplot.legend()
# show the plot
pyplot.show()

In [61]:
y_pred = model.predict(x_test)

/home/deepracer/MADS-Medicare-Project/venv/lib/python3.8/site-packages/xgboost/data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


In [62]:
full_test['state'] = full_test.reset_index()['idx'].apply(lambda x: x.split("_")[0]).values

In [63]:
def perc_round(x):
    return round(x,-2)

full_test['pred'] = np.array(list(map(math.exp,y_pred)))
full_test['target_original'] = np.array(list(map(math.exp,full_test['target'])))
full_test['total_diff'] = np.array(list(map(round, full_test['target_original'] - full_test['pred'])))
full_test['perc_diff'] = full_test['total_diff'].values/full_test['target_original'].values

In [64]:
full_test.sample(15)

,target,1,2,3,4,0_1,0_2,0_3,0_4,1_1,1_2,1_3,1_4,state,pred,target_original,total_diff,perc_diff
idx,,,,,,,,,,,,,,,,,,
Michigan_25445,0.000000,0.000000,0.000000,0.000000,2.639057,0.575498,0.571357,0.568785,0.569463,0.452120,0.448292,0.443766,0.440798,Michigan,1.892754,1.0,-1,-1.000000
Hawaii_74328,5.365976,5.283204,5.356586,4.859812,4.804021,-0.574385,-0.572939,-0.573450,-0.575063,-0.652140,-0.653734,-0.655994,-0.657801,Hawaii,183.084811,214.0,31,0.144860
Wisconsin_37183,3.091042,2.833213,2.564949,2.890372,2.772589,-0.030085,-0.031952,-0.032171,-0.033151,-0.112231,-0.116293,-0.120374,-0.123075,Wisconsin,9.567136,22.0,12,0.545455
Colorado_86146,7.958227,7.892452,7.866722,7.811163,7.717351,0.018745,0.007310,-0.006639,-0.024039,-0.190727,-0.200887,-0.212898,-0.224931,Colorado,2040.817355,2859.0,818,0.286114
Minnesota_67036,5.351858,5.123964,5.117994,4.969813,5.117994,-0.024673,-0.032363,-0.039062,-0.044455,-0.170073,-0.175552,-0.181244,-0.185465,Minnesota,167.484068,211.0,44,0.208531
Washington_93600,3.850148,3.784190,2.397895,2.772589,2.995732,0.311310,0.287503,0.263717,0.244507,0.062861,0.047921,0.031864,0.017852,Washington,24.344148,47.0,23,0.489362
Mississippi_99201,8.197814,8.231908,8.277412,8.491465,8.587279,-0.421682,-0.420003,-0.418299,-0.416519,-0.500592,-0.502625,-0.504733,-0.506360,Mississippi,2821.298413,3633.0,812,0.223507
New Jersey_22551,6.834109,6.854355,6.852243,6.668228,6.642487,0.406448,0.404752,0.407474,0.409549,0.312633,0.309062,0.304248,0.299973,New Jersey,903.371815,929.0,26,0.027987
Washington_86022,3.737670,4.060443,3.951244,4.343805,4.343805,0.311310,0.287503,0.263717,0.244507,0.062861,0.047921,0.031864,0.017852,Washington,55.283634,42.0,-13,-0.309524


In [65]:
full_test.groupby(['state']).mean()

,target,1,2,3,4,0_1,0_2,0_3,0_4,1_1,1_2,1_3,1_4,pred,target_original,total_diff,perc_diff
state,,,,,,,,,,,,,,,,,
Alabama,4.731577,4.765754,4.876559,4.929091,4.888554,-0.162520,-0.162634,-0.162152,-0.162000,-0.236737,-0.240379,-0.244837,-0.248653,9377.318793,12181.742127,2804.425100,-164.661361
Alaska,4.016383,3.912151,3.786894,3.753294,3.586644,-0.738980,-0.738410,-0.739370,-0.739752,-0.786242,-0.786586,-0.787076,-0.787268,1393.908653,1963.029078,569.093972,-29.116408
American Samoa,3.545848,3.651175,3.699190,3.417921,3.271268,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,182.202200,242.777778,60.555556,-0.483398
Arizona,5.010179,4.913235,4.866397,4.825870,4.738133,0.207604,0.196971,0.184604,0.173674,0.015564,-0.003810,-0.023994,-0.042012,11022.700882,13864.593295,2841.898975,-110.701721
Arkansas,4.516073,4.454019,4.460854,4.527186,4.479630,-0.422374,-0.423939,-0.425101,-0.426399,-0.489533,-0.491847,-0.494542,-0.496743,7332.302718,8524.247319,1191.965017,-119.288832
California,5.594946,5.534571,5.500199,5.449106,5.417563,5.445280,5.431557,5.412786,5.376525,3.914986,3.867405,3.809419,3.748269,23188.051404,46666.585341,23478.522122,-426.846031
Colorado,4.699988,4.624256,4.576409,4.554724,4.510389,0.018745,0.007310,-0.006639,-0.024039,-0.190727,-0.200887,-0.212898,-0.224931,6163.677510,8112.530875,1948.872366,-96.936728
Connecticut,4.324508,4.406576,4.453731,4.625364,4.607304,-0.366636,-0.366110,-0.364265,-0.362967,-0.383384,-0.383339,-0.383399,-0.383345,5588.659444,5869.925350,281.271807,-114.033397
Delaware,4.199545,4.174130,4.110285,4.128022,4.057584,-0.733901,-0.734477,-0.734897,-0.735719,-0.741599,-0.743420,-0.745366,-0.747187,2213.317161,3519.645469,1306.321781,-59.606110


In [23]:
excl_states =  ['Armed Forces Central/South America', 'Armed Forces Europe', 'Armed Forces Pacific', 'Foreign Country']

full_test.loc[~full_test['state'].isin(excl_states),:]

,target,1,2,3,4,0_1,0_2,0_3,0_4,1_1,1_2,1_3,1_4,state,pred,target_original,total_diff,perc_diff
idx,,,,,,,,,,,,,,,,,,
Alabama_00100,5.662960,5.690359,5.828946,5.899897,5.937536,-0.162520,-0.162634,-0.162152,-0.162000,-0.236737,-0.240379,-0.244837,-0.248653,Alabama,254.757788,288.0,33,0.114583
Alabama_00102,0.000000,0.000000,0.000000,5.036953,0.000000,-0.162520,-0.162634,-0.162152,-0.162000,-0.236737,-0.240379,-0.244837,-0.248653,Alabama,3.604539,1.0,-3,-3.000000
Alabama_00103,8.044305,8.012350,8.250620,8.189522,8.248006,-0.162520,-0.162634,-0.162152,-0.162000,-0.236737,-0.240379,-0.244837,-0.248653,Alabama,2567.173238,3116.0,549,0.176187
Alabama_00104,7.426549,7.807917,8.270269,8.133881,8.055475,-0.162520,-0.162634,-0.162152,-0.162000,-0.236737,-0.240379,-0.244837,-0.248653,Alabama,1790.662938,1680.0,-111,-0.066071
Alabama_00120,6.602588,6.708084,6.768493,6.687109,6.666957,-0.162520,-0.162634,-0.162152,-0.162000,-0.236737,-0.240379,-0.244837,-0.248653,Alabama,719.656128,737.0,17,0.023066
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Wyoming_Q5101,11.523865,10.509442,0.000000,0.000000,0.000000,-0.783313,-0.781783,-0.781225,-0.781926,-0.798589,-0.798755,-0.799054,-0.799364,Wyoming,9327.633149,101100.0,91772,0.907735
Wyoming_Q9957,4.330733,4.430817,3.401197,2.833213,0.000000,-0.783313,-0.781783,-0.781225,-0.781926,-0.798589,-0.798755,-0.799054,-0.799364,Wyoming,48.499161,76.0,28,0.368421
Wyoming_Q9965,3.433987,3.583519,4.553877,4.418841,4.709530,-0.783313,-0.781783,-0.781225,-0.781926,-0.798589,-0.798755,-0.799054,-0.799364,Wyoming,24.655398,31.0,6,0.193548


In [24]:
full_test.sort_values('pred',ascending=False)

,target,1,2,3,4,0_1,0_2,0_3,0_4,1_1,1_2,1_3,1_4,state,pred,target_original,total_diff,perc_diff
idx,,,,,,,,,,,,,,,,,,
California_Q9967,16.107760,16.059161,16.023511,15.941482,15.959088,5.445280,5.431557,5.412786,5.376525,3.914986,3.867405,3.809419,3.748269,California,7.873626e+06,9897179.7,2023554,0.204458
California_99213,15.977317,16.004393,16.029845,16.044195,16.066907,5.445280,5.431557,5.412786,5.376525,3.914986,3.867405,3.809419,3.748269,California,7.873626e+06,8686818.5,813192,0.093612
Texas_Q9967,16.030637,16.039197,16.019768,15.991413,16.019904,3.765884,3.715102,3.650929,3.569424,2.305769,2.248578,2.185494,2.123262,Texas,7.493917e+06,9162566.3,1668649,0.182116
Texas_A0425,16.276326,16.325137,16.295087,16.282559,16.187829,3.765884,3.715102,3.650929,3.569424,2.305769,2.248578,2.185494,2.123262,Texas,7.493917e+06,11714356.3,4220439,0.360279
Florida_Q0138,16.284928,16.119656,16.156415,15.963191,15.935271,1.995818,1.953507,1.903172,1.856782,2.066368,2.009058,1.943417,1.885768,Florida,7.205057e+06,11815556.0,4610499,0.390206
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Mississippi_J9351,0.000000,0.000000,8.173011,0.000000,8.789965,-0.421682,-0.420003,-0.418299,-0.416519,-0.500592,-0.502625,-0.504733,-0.506360,Mississippi,2.744787e-01,1.0,1,1.000000
Tennessee_83789,4.394449,0.000000,6.898715,12.171911,12.504858,0.118073,0.107946,0.100679,0.094020,-0.006950,-0.015463,-0.024697,-0.032966,Tennessee,2.473220e-01,81.0,81,1.000000
Oklahoma_J9178,0.000000,0.000000,7.495542,0.000000,7.895436,-0.226620,-0.226902,-0.229508,-0.236450,-0.382753,-0.385034,-0.388473,-0.391567,Oklahoma,2.299776e-01,1.0,1,1.000000


In [25]:
del(df_geo)
del(df_geo_features)
del(df_geo_features_test)
del(df_geo_features_train)
del(df_geo_full)
del(pop_agg_df)
del(pop_test_df)
del(pop_train_df)


In [ ]:
from hyperopt import hp
from hyperopt import fmin, tpe, STATUS_OK, STATUS_FAIL, Trials
from sklearn.model_selection import cross_val_score
num_estimator = [100,150,200,250,500,1000,2000,5000,10000]
import xgboost as xgb

space=  {'max_depth': hp.quniform("max_depth", 3, 18, 1),
        'gamma': hp.uniform ('gamma', 1,9),
        'reg_alpha' : hp.quniform('reg_alpha', 30,180,1),
        'reg_lambda' : hp.uniform('reg_lambda', 0,1),
        'colsample_bytree' : hp.uniform('colsample_bytree', 0.5,1),
        'min_child_weight' : hp.quniform('min_child_weight', 0, 10, 1),
        'n_estimators': hp.choice("n_estimators", num_estimator),
    }

def hyperparameter_tuning(space):
    model=xgb.XGBRegressor(n_estimators = space['n_estimators'], max_depth = int(space['max_depth']), gamma = space['gamma'],
                         reg_alpha = int(space['reg_alpha']) , min_child_weight=space['min_child_weight'],
                         colsample_bytree=space['colsample_bytree'], objective="reg:squarederror")
    
    score_cv = cross_val_score(model, x_train, y_train, cv=5, scoring="neg_mean_absolute_error").mean()
    return {'loss':-score_cv, 'status': STATUS_OK, 'model': model}


trials = Trials()
best = fmin(fn=hyperparameter_tuning,
            space=space,
            algo=tpe.suggest,
            max_evals=200,
            trials=trials)

print(best)

  0%|                                                                           | 0/200 [00:00<?, ?trial/s, best loss=?]

/home/deepracer/MADS-Medicare-Project/venv/lib/python3.8/site-packages/xgboost/data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):

/home/deepracer/MADS-Medicare-Project/venv/lib/python3.8/site-packages/xgboost/data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):

/home/deepracer/MADS-Medicare-Project/venv/lib/python3.8/site-packages/xgboost/data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):

/home/deepracer/MADS-Medicare-Project/venv/lib/python3.8/site-packa

  0%|▏                                              | 1/200 [00:44<2:25:59, 44.02s/trial, best loss: 0.6135579100582856]

/home/deepracer/MADS-Medicare-Project/venv/lib/python3.8/site-packages/xgboost/data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):

/home/deepracer/MADS-Medicare-Project/venv/lib/python3.8/site-packages/xgboost/data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):

/home/deepracer/MADS-Medicare-Project/venv/lib/python3.8/site-packages/xgboost/data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):

/home/deepracer/MADS-Medicare-Project/venv/lib/python3.8/site-packa

  1%|▍                                             | 2/200 [03:19<6:01:37, 109.59s/trial, best loss: 0.6135579100582856]

...

/home/deepracer/MADS-Medicare-Project/venv/lib/python3.8/site-packages/xgboost/data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):

/home/deepracer/MADS-Medicare-Project/venv/lib/python3.8/site-packages/xgboost/data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):

/home/deepracer/MADS-Medicare-Project/venv/lib/python3.8/site-packages/xgboost/data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):

/home/deepracer/MADS-Medicare-Project/venv/lib/python3.8/site-packa

 10%|███▉                                      | 19/200 [2:10:51<10:57:36, 217.99s/trial, best loss: 0.6099345621028881]

/home/deepracer/MADS-Medicare-Project/venv/lib/python3.8/site-packages/xgboost/data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):

/home/deepracer/MADS-Medicare-Project/venv/lib/python3.8/site-packages/xgboost/data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):

/home/deepracer/MADS-Medicare-Project/venv/lib/python3.8/site-packages/xgboost/data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):

/home/deepracer/MADS-Medicare-Project/venv/lib/python3.8/site-packa

 10%|████▏                                     | 20/200 [2:17:41<13:46:52, 275.62s/trial, best loss: 0.6099345621028881]

/home/deepracer/MADS-Medicare-Project/venv/lib/python3.8/site-packages/xgboost/data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):



In [30]:
best

{'colsample_bytree': 0.9682038820754064,
 'gamma': 1.0184993645158578,
 'max_depth': 12.0,
 'min_child_weight': 8.0,
 'n_estimators': 6,
 'reg_alpha': 113.0,
 'reg_lambda': 0.08792023426025397}

{'colsample_bytree': 0.9682038820754064,
 'gamma': 1.0184993645158578,
 'max_depth': 12.0,
 'min_child_weight': 8.0,
 'n_estimators': 6,
 'reg_alpha': 113.0,
 'reg_lambda': 0.08792023426025397}

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=f3594ae2-e1b0-4476-8420-ffda39dfce6e' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>